In [7]:
import pandas as pd

import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [8]:
df = pd.DataFrame()
driver = webdriver.Chrome()

for page in range(1,6):
    print(f"Página {page}")

    url = "https://www.olx.com.br/imoveis/venda/estado-pa?q=belém&o={}".format(page)
    driver.get(url)
    time.sleep(5)
    elements_list = driver.find_elements(By.CSS_SELECTOR,"#main-content > div.sc-74d68375-2.cBiAyi > div")
    print(f"início do laço com {len(elements_list)}")

    for i,element in enumerate(elements_list):
        # definindo a serie
        series = pd.Series()

        #move to element to render
        actions = ActionChains(driver)
        actions.move_to_element(element).perform()
        classes = element.get_attribute("class").split(' ')
        if "sc-74d68375-0" not in classes:
            continue
        
        # Nome do anúncio
        name = element.find_element(By.CLASS_NAME,"olx-ad-card__title--horizontal")
        series['name'] = name.text

        # Preço de venda
        prices = element.find_element(By.CLASS_NAME,"olx-ad-card__details-price--horizontal")
        main_price = prices.find_element(By.CSS_SELECTOR,"h3")
        series['main_price'] = main_price.text

        tax_prices = prices.find_elements(By.CSS_SELECTOR,"p")
        for price in tax_prices:
            price_var = price.text
            if "Condom" in price_var:
                series['Condominio'] = price_var
            elif "IPTU" in price_var:
                series['IPTU'] = price_var
            else:
                series['Old price'] = price_var

        # Localização
        local = element.find_element(By.CLASS_NAME,"olx-ad-card__location-date-container")
        series['local'] = local.text

        # Data de publicação
        publish_date = element.find_element(By.CLASS_NAME,"olx-ad-card__date--horizontal")
        series['date'] = publish_date.text

        # Comodidades
        info = element.find_element(By.CLASS_NAME,"olx-ad-card__labels-items")
        spans = info.find_elements(By.CSS_SELECTOR,"span")
        
        for span in spans:
            info_aria_label = span.get_attribute("aria-label")

            if 'banheiro' in info_aria_label:
                series['banheiro']  = info_aria_label
            elif 'metro' in info_aria_label:
                series['metros m²']  = info_aria_label
            elif 'garagem' in info_aria_label:
                series['garagem']  = info_aria_label
            elif 'quarto' in info_aria_label:
                series['quarto']  = info_aria_label
            else:
                raise Exception("Erro no campo {} item {}".format(info_aria_label,i))

        df = pd.concat([df, pd.DataFrame([series])])

Página 1
início do laço com 55


C:\Users\Usuário\AppData\Local\Temp\ipykernel_3704\931598781.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series = pd.Series()


Página 2
início do laço com 55
Página 3
início do laço com 55
Página 4
início do laço com 55
Página 5
início do laço com 55


In [9]:
df

,name,main_price,Condominio,local,date,quarto,metros m²,garagem,banheiro,IPTU,Old price
0,BELéM - Apartamento Padrão - Nazaré,R$ 2.000.000,Condomínio R$ 2.100,"Belém, Nazaré","Ontem, 09:13",4 quartos,288 metros quadrados,3 vagas de garagem,4 banheiros,NaN,NaN
0,BELEM - Apartamento Padrão - São Brás,R$ 940.844,NaN,"Belém, São Brás","Ontem, 03:59",3 quartos,123 metros quadrados,2 vagas de garagem,2 banheiros,NaN,NaN
0,BELéM - Apartamento Padrão - Cremação,R$ 420.000,Condomínio R$ 900,"Belém, Guamá","16 de fev, 03:51",3 quartos,90 metros quadrados,1 vaga de garagem,3 banheiros,NaN,NaN
0,BELéM - Apartamento Padrão - Nazaré,R$ 5.500.000,NaN,"Belém, Nazaré","16 de fev, 03:13",4 quartos,701 metros quadrados,5 ou mais vagas de garagem,5 ou mais banheiros,NaN,NaN
0,BELéM - Apartamento Padrão - Nazare,R$ 1.600.000,Condomínio R$ 1.300,"Belém, Nazaré","15 de fev, 12:09",3 quartos,276 metros quadrados,2 vagas de garagem,3 banheiros,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
0,Bela casa no distrito industrial 140,R$ 140,NaN,"Ananindeua, Distrito Industrial","10 de fev, 16:58",2 quartos,12 metros quadrados,3 vagas de garagem,2 banheiros,IPTU R$ 100,NaN
0,"belo apto com fino acabamento, 3 suítes com 20...",R$ 1.000.000,Condomínio R$ 1.350,"Belém, Nazaré","9 de fev, 09:51",4 quartos,200 metros quadrados,2 vagas de garagem,5 ou mais banheiros,IPTU R$ 4.800,NaN
0,Bela Vida II - Apartamento de 2 Quartos - Fina...,R$ 250.000,Condomínio R$ 300,"Belém, Tapanã (Icoaraci)","9 de fev, 01:13",2 quartos,45 metros quadrados,1 vaga de garagem,2 banheiros,IPTU R$ 96,NaN
0,Bela Vida II - Apartamento de 2 Quartos - Fina...,R$ 250.000,Condomínio R$ 300,"Belém, Tapanã (Icoaraci)","9 de fev, 00:51",2 quartos,45 metros quadrados,1 vaga de garagem,2 banheiros,IPTU R$ 96,NaN
